In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/books-movies-reviews/books_films_reviews.sqlite3


In [2]:
from nltk import tokenize, word_tokenize
from nltk.corpus import stopwords
import re

In [3]:
STOP_WORDS = stopwords.words('english')

The goal is to summarize each document by finding the sentences with the highest combindation of word weights.  The basic idea comes from here:  https://towardsdatascience.com/summarize-a-text-with-python-b3b260c60e72

In [4]:
# Load the data set:
import sqlite3

# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("/kaggle/input/books-movies-reviews/books_films_reviews.sqlite3")
df = pd.read_sql_query("SELECT * from book_reviews order by book_id", con)
df.head()

,book_id,review_id,rating,review_text,language,id,primary_topic_id,primary_topic_prob,rating_numeric
0,1,829021593,it was amazing,"""Read Harry Potter!"" they said. ""It'll be fun!...",en,177953,16.0,0.380909,5.0
1,1,737648743,it was amazing,i dont want to talk to anyonedont even look at...,en,177954,16.0,0.440062,5.0
2,1,683662307,it was amazing,(A) 86% | ExtraordinaryNotes: It dwells on the...,en,177955,1.0,0.373827,5.0
3,1,137941096,it was amazing,This remains one of my top favorite HP books! ...,en,177956,2.0,0.299372,5.0
4,1,2856577,it was amazing,I was first introduced to Harry Potter in a Ch...,en,177957,2.0,0.324355,5.0


In [5]:
def summarize_doc(doc, num_sentences = 3):
    # handle cases where there is no space after a period: e.g. " end.But "
    doc = re.sub(r'([a-z])\.([A-Z])', r'\1. \2', doc)
    word_weights={}
    for word in word_tokenize(doc) :  
        word = word.lower()
        if len(word) > 1 and word not in STOP_WORDS:
            if word in word_weights.keys():            
                word_weights[word] += 1
            else:
                word_weights[word] = 1
    #print(word_weights)
    sentence_weights={}
    min_weight = 0
    for sent in tokenize.sent_tokenize(doc):
        sentence_weight = 0
        count = 0
        for word in word_tokenize(sent) :  
            word = word.lower()
            count += 1
            if word in word_weights.keys():
                sentence_weight += word_weights[word]
        #print (sentence_weight, sent)
        sentence_weight = sentence_weight * 1.0 / count # avg
        if len(sentence_weights) < num_sentences:
            sentence_weights[sent] = sentence_weight
            min_weight = min(sentence_weights.values())
        else:
            if sentence_weight > min_weight:
                #print(sentence_weight, min_weight)
                # switch out current and previous min
                for k,v in sentence_weights.items():
                    if v == min_weight:
                        sentence_weights.pop(k)
                        sentence_weights[sent] = sentence_weight
                        break
                min_weight = min(sentence_weights.values())
    print(sentence_weights)
    #highest_weights = sorted(sentence_weights.values(),reverse=True)[0:num_sentences]
    return ' '.join(sentence_weights.keys())

In [6]:
text = df.iloc[990]["review_text"]
text

"It has taken almost seven months for me to read LotR aloud to my two daughters; we felt a sense of achievement and sadness when we finished tonight. The last time I read the novel was just prior to the first of Peter Jackson's films. I did this consciously fearing that after the films the story would never feel the same again. Reading the book aloud made me appreciate Tolkien's love of landscape, trees, plants, gardens, mountains and natural features of all kind. The extensive use of geographic terms was particularly notable. There are many, many memorable moments in the novel but our favourites, in no particular order: meeting Strider in the Prancing Pony; Sam singing 'The Lay of Gilgalad'; Frodo being wounded on Weathertop by the Black Riders; meeting Gollum; Aragorn, Legolas and Gimli's long journey chasing the orcs who had kidnapped Merry and Pippin; meeting Treebeard; when Gandalf fights the Balrog; and any time Galadriel was present, especially with her mirror. We really liked F

In [7]:
summarize_doc(text)

{"The last time I read the novel was just prior to the first of Peter Jackson's films.": 0.8947368421052632, "Reading the book aloud made me appreciate Tolkien's love of landscape, trees, plants, gardens, mountains and natural features of all kind.": 0.7037037037037037, "There are many, many memorable moments in the novel but our favourites, in no particular order: meeting Strider in the Prancing Pony; Sam singing 'The Lay of Gilgalad'; Frodo being wounded on Weathertop by the Black Riders; meeting Gollum; Aragorn, Legolas and Gimli's long journey chasing the orcs who had kidnapped Merry and Pippin; meeting Treebeard; when Gandalf fights the Balrog; and any time Galadriel was present, especially with her mirror.": 0.6744186046511628}


"The last time I read the novel was just prior to the first of Peter Jackson's films. Reading the book aloud made me appreciate Tolkien's love of landscape, trees, plants, gardens, mountains and natural features of all kind. There are many, many memorable moments in the novel but our favourites, in no particular order: meeting Strider in the Prancing Pony; Sam singing 'The Lay of Gilgalad'; Frodo being wounded on Weathertop by the Black Riders; meeting Gollum; Aragorn, Legolas and Gimli's long journey chasing the orcs who had kidnapped Merry and Pippin; meeting Treebeard; when Gandalf fights the Balrog; and any time Galadriel was present, especially with her mirror."

This code has a flaw that longer sentences always have more weight than shorter ones.  This may be solved by using tf-idf with a parameter to not reward extra words.